In [ ]:
!pip install gliner

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset, Dataset
import torch
torch.cuda.empty_cache()

In [ ]:
# CHUNKED_K_FOLDS/123_4_5/test.jsonl
BUSTER_BIO_CHUNKED = load_dataset(path='json', data_files='./test.json')['train']

In [ ]:
print(BUSTER_BIO_CHUNKED)

Dataset({
    features: ['document_id', 'tokens', 'labels', 'positions', 'chunk_id'],
    num_rows: 1942
})


In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_large", map_location='cuda')

In [ ]:
labels_option = ['buying company', 'selling company', 'acquired company', 'generic consulting company', 'legal consulting company', 'annual revenues']

In [ ]:
predictions = []
batch_size = 32
BUSTER_BIO_CHUNKED = BUSTER_BIO_CHUNKED.to_list()
for i in range(0, len(BUSTER_BIO_CHUNKED), batch_size):
  input_batch = []
  for chunk_sample in BUSTER_BIO_CHUNKED[i:i+batch_size]:
    input = ' '.join(chunk_sample['tokens'])
    input_batch.append(input)
  batch_predictions = model.batch_predict_entities(input_batch, labels_option)
  for chunk_sample, pred in zip(BUSTER_BIO_CHUNKED[i:i+batch_size], batch_predictions):
    predictions.append({
      'document_id': chunk_sample['document_id'],
      'chunk_id': chunk_sample['chunk_id'],
      'tokens': chunk_sample['tokens'],
      'labels': chunk_sample['labels'],
      'prediction': pred
  })

In [ ]:
predictions = []
for chunk_sample in BUSTER_BIO_CHUNKED:
  input = ' '.join(chunk_sample['tokens'])
  entities = model.predict_entities(input, labels_option)
  predictions.append({
      'document_id': chunk_sample['document_id'],
      'chunk_id': chunk_sample['chunk_id'],
      'tokens': chunk_sample['tokens'],
      'labels': chunk_sample['labels'],
      'prediction': entities
  })

In [ ]:
print(predictions)

In [ ]:
predictions_Dataset = Dataset.from_list(predictions)
predictions_Dataset.to_json("./GLiNER_large_on_BUSTER_chunked_123_4_5.jsonl")

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

8941041

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/GLiNER_large_on_BUSTER_chunked_123_4_5.jsonl"
predictions_Dataset.to_json(file_path)